# Import and Set the Default Random Seed

In [2]:
#All Imported Packages
import os
import time
import numpy as np
import pandas as pd
from math import log
from tqdm import tqdm
from PIL import Image
from collections import Counter
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from scipy.cluster.vq import kmeans2
from sklearn.metrics import accuracy_score
from scipy.optimize import linear_sum_assignment
from sklearn.metrics import normalized_mutual_info_score

# Default Random Seeds
np.random.seed(42)

# Define Data Loading

In [3]:
def load_data(root='data/CroppedYaleB', reduce=4):
    """ 
    Load ORL (or Extended YaleB) dataset to numpy array.
    
    Args:
        root: path to dataset.
        reduce: scale factor for zooming out images.
        
    """ 
    images, labels = [], []

    for i, person in enumerate(sorted(os.listdir(root))):
        
        if not os.path.isdir(os.path.join(root, person)):
            continue
        
        for fname in os.listdir(os.path.join(root, person)):    
            
            if fname.endswith('Ambient.pgm'):
                continue
            
            if not fname.endswith('.pgm'):
                continue
                
            img = Image.open(os.path.join(root, person, fname))
            img = img.convert('L') 

            img = img.resize([s//reduce for s in img.size])

            img  = np.asarray(img, dtype='float32')
            # Normalize pixel values to [0,1]
            img = img / 255

            # Mean centering or variance scaling
            img /= np.linalg.norm(img) + 1e-8

            # convert image to numpy array.
            img = img.reshape((-1,1))

            # collect data and label.
            images.append(img)
            labels.append(i)

    # concate all images and labels.
    images = np.concatenate(images, axis=1)
    labels = np.array(labels)

    return images, labels

# Define Salt & Pepper Noise Function

In [4]:
def salt_pepper(X, p=0.4, r=0.3):
    """
    Add salt-and-pepper noise to an image matrix (X) ensuring the total
    proportion of corrupted pixels is exactly 'p'.

    Args:
        X (np.ndarray): Original image matrix with values in [0,1].
        p (float): Total proportion of pixels to corrupt (p).
        r (float): Fraction of corrupted pixels to set as white (salt).

    Returns:
        X_noisy (np.ndarray): Noisy image matrix.
        noise (np.ndarray): Noise matrix (1.0 for salt, -1.0 for pepper, 0.0 otherwise).
    """
        
    X_noisy = X.copy()
    noise = np.zeros_like(X, dtype=np.float32)
    
    num_pixels = X_noisy.size
    num_noise = int(num_pixels * p)

    if num_noise == 0:
        return X_noisy, noise

    all_noise_coord = np.random.choice(num_pixels, num_noise, replace=False)
    
    num_salt = int(num_noise * r)
    num_pepper = num_noise - num_salt
    
    np.random.shuffle(all_noise_coord)
    salt_coord = all_noise_coord[:num_salt]
    pepper_coord = all_noise_coord[num_salt:]

    X_noisy.flat[salt_coord] = 1.0
    noise.flat[salt_coord] = 1.0
    
    X_noisy.flat[pepper_coord] = 0.0
    noise.flat[pepper_coord] = -1.0
        
    return X_noisy, noise

In [5]:
def load_data_salt_pepper(root='data/CroppedYaleB', reduce=4, p=0, r=0):
    """ 
    Load ORL (or Extended YaleB) dataset to numpy array.
    
    Args:
        root: path to dataset.
        reduce: scale factor for zooming out images.
        
    """ 
    images, labels = [], []

    for i, person in enumerate(sorted(os.listdir(root))):
        
        if not os.path.isdir(os.path.join(root, person)):
            continue
        
        for fname in os.listdir(os.path.join(root, person)):    
            
            # Remove background images in Extended YaleB dataset.
            if fname.endswith('Ambient.pgm'):
                continue
            
            if not fname.endswith('.pgm'):
                continue
                
            # load image.
            img = Image.open(os.path.join(root, person, fname))
            img = img.convert('L') # grey image.

            # reduce computation complexity.
            img = img.resize([s//reduce for s in img.size])

            # Convert to float
            img  = np.asarray(img, dtype='float32')
            img /= 255.0 
            
            # apply salt and pepper BEFORE L2 normalization
            img, noise_mask = salt_pepper(img, p=p, r=r)
            
            # then L2-normalize
            img /= (np.linalg.norm(img) + 1e-8)
            
            # convert image to numpy array.
            img = img.reshape((-1,1))

            # collect data and label.
            images.append(img)
            labels.append(i)

    # concate all images and labels.
    images = np.concatenate(images, axis=1)
    labels = np.array(labels)

    return images, labels

# Define NMF Algorithms

## L2-Norm Based Non-negative Matrix Factorization (L2-Norm NMF)

In [7]:
def L2_nmf(X, K, lr=0.001, steps=5000, tol=1e-2, verbose = True):
    """
    Non-negative Matrix Factorization (NMF) for 
    the basic L2 (Frobenius) objective function.

    Parameters:
    1. X: Input matrix (M x N), must be non-negative.
    2. K: Rank of factorization (number of latent features).
    3. steps: Maximum number of iterations.
    4. tol: Tolerance for early stopping based on error change.
    5. verbose: If True, prints error every 10 steps.
    
    Returns:
    W: Basis matrix (M x K).
    H: Coefficient matrix (K x N).
    WH: The final reconstructed matrix W @ H.
    """
    M , N = X.shape
    # X = X / np.linalg.norm(X, "fro")
    W = np.random.rand(M, K) * np.sqrt(np.mean(X))
    H = np.random.rand(K, N) * np.sqrt(np.mean(X))

    tol = tol
    prev_e=0
    
    for step in range(steps):
        WH = W @ H
        e = np.linalg.norm(X - WH, "fro")**2

        dW = -2 * X @ H.T + 2 * W @ (H @ H.T)
        dH = -2 * W.T @ X + 2 * (W.T @ W) @ H

        W -= lr * dW
        H -= lr * dH

        # enforce non-negativity
        W = np.maximum(W, 0)
        H = np.maximum(H, 0)

        if step % 10 == 0:
            print(f'step:{step}, e:{e:.4f}')
        if step > 0:
            rel_change = abs(e - prev_e) / (prev_e + 1e-12)
            if rel_change < tol:   
                print(f'Converged at step {step}, e = {e:.4f}, rel_change={rel_change:.3e}')
                break
        prev_e = e

    return W, H, WH, step+1

## Lee--Seung Multiplicative Update NMF (MU-L2)

In [8]:
def L2Lee_Seung(X, K, steps=10000, tol=1e-2, epsilon=1e-9, verbose=True):
    """
    Non-negative Matrix Factorization (NMF) using Lee & Seung's 
    Multiplicative Update (MU) rules for the L2 (Frobenius) objective function.

    Parameters:
    1. X: Input matrix (M x N), must be non-negative.
    2. K: Rank of factorization (number of latent features).
    3. steps: Maximum number of iterations.
    4. tol: Tolerance for early stopping based on error change.
    5. verbose: If True, prints error every 10 steps.
    
    Returns:
    W: Basis matrix (M x K).
    H: Coefficient matrix (K x N).
    reconstruction: The final reconstructed matrix W @ H.
    """
    M, N = X.shape
    avg_X = np.mean(X)
    W = np.random.rand(M, K) * np.sqrt(avg_X)
    H = np.random.rand(K, N) * np.sqrt(avg_X)

    prev_e = 0
    epsilon = epsilon
    for step in range(steps):
        WH = W @ H
        e = np.linalg.norm(X - WH, "fro")**2

        numerator_H = W.T @ X
        denominator_H = W.T @ W @ H

        epsilon = epsilon         
        H *= (numerator_H / (denominator_H + epsilon))

        numerator_W = X @ H.T
        denominator_W = W @ H @ H.T
        W *= (numerator_W / (denominator_W + epsilon))

        
        if verbose and step % 10 == 0:
            print(f'step:{step}, e:{e:.4f}')
            
        if step > 0:
            rel_change = abs(e - prev_e) / (prev_e + 1e-12)
            if rel_change < tol:   
                print(f'Converged at step {step}, e = {e:.4f}, rel_change={rel_change:.3e}')
                break
        prev_e = e

    return W, H, WH, step+1

## L1-Norm Regularized Robust NMF

In [9]:
def l1_nmf_corrected(X, rank=40, steps=500, tol=1e-4, 
                     lambda_h=0.0, lambda_w=0.0, eps=1e-8):
    m, n = X.shape
    norm_X = np.linalg.norm(X)
    lambda_h = lambda_h / norm_X
    lambda_w = lambda_w / norm_X
    
    W = np.abs(np.random.randn(m, rank)).astype(np.float32)
    H = np.abs(np.random.randn(rank, n)).astype(np.float32)
    
    # 外层IRLS迭代（权重更新）
    for irls_iter in range(10):  # 通常5-10次IRLS迭代
        # 1. 计算当前重构和残差（IRLS外层固定权重）
        WH = W @ H
        E_mat = X - WH
        weight = 1.0 / np.sqrt(E_mat**2 + eps)  # 权重仅在此更新
        
        # 2. 内层NMF迭代（固定权重求解加权L2）
        prev_E = np.inf
        for step in range(steps):
            # 更新H（带正则化）
            num_H = W.T @ (weight * X)
            den_H = W.T @ (weight * WH)
            H *= num_H / (den_H + lambda_h + eps)  # 同时考虑H正则化
            
            # 更新W（带正则化）
            WH = W @ H
            num_W = (weight * X) @ H.T
            den_W = (weight * WH) @ H.T
            W *= num_W / (den_W + lambda_w + eps)  # 同时考虑W正则化
            
            # 收敛检查（使用加权Frobenius误差）
            weighted_error = weight * (X - W @ H)
            E_sq = np.sum(weighted_error ** 2)
            if abs(E_sq - prev_E) / (prev_E + 1e-12) < tol:
                break
            prev_E = E_sq
    
    return W, H, W @ H, step + 1

## Hypersurface Cost-Based NMF (HCNMF)

In [10]:
def hypersurface_nmf(X, rank=30, steps=100, tol=1e-3, delta=0.1, eps=1e-8, verbose=False):
    """
    Corrected Hypersurface Cost-based NMF:
        f(E) = sum( sqrt(1 + (E/delta)^2) - 1 )
    """
    m, n = X.shape
    W = np.abs(np.random.randn(m, rank)).astype(np.float32)
    H = np.abs(np.random.randn(rank, n)).astype(np.float32)
    
    # 初始化超曲面代价
    prev_cost = float('inf')
    
    for step in range(steps):
        WH = W @ H
        E = X - WH
        
        # 关键修复1: 正确计算权重
        denom = np.sqrt(delta**2 + E**2)  # 注意：分母是sqrt(δ² + E²)
        weights = 1.0 / (denom + eps)     # 避免除零
        
        # 更新H (使用标准加权NMF规则)
        num_H = W.T @ (X * weights)
        den_H = W.T @ (WH * weights) + eps
        H *= num_H / den_H
        
        # 更新W (注意：使用更新后的H)
        WH = W @ H
        E = X - WH
        denom = np.sqrt(delta**2 + E**2)
        weights = 1.0 / (denom + eps)
        
        num_W = (X * weights) @ H.T
        den_W = (WH * weights) @ H.T + eps
        W *= num_W / den_W
        
        # 非负投影
        W = np.clip(W, eps, None)
        H = np.clip(H, eps, None)
        
        # 关键修复2: 使用超曲面代价判断收敛
        current_cost = np.sum(np.sqrt(1 + (X - W @ H)**2 / delta**2) - 1)
        
        if verbose and (step % max(1, steps//10) == 0 or step == steps-1):
            print(f"step {step+1}/{steps} | Cost={current_cost:.6f} | Δ={abs(current_cost - prev_cost):.3e}")
        
        # 检查收敛
        if step > 0:
            rel_change = abs(current_cost - prev_cost) / (prev_cost + 1e-12)
            if rel_change < tol:
                if verbose:
                    print(f'Converged at step {step} | Cost={current_cost:.4f} | rel_change={rel_change:.3e}')
                break
        
        prev_cost = current_cost

    return W, H, W @ H, step + 1

## Deep NMF: Stacked NMF

In [11]:
import numpy as np
def nmf_mu(X, K, steps=200, eps=1e-9, tol=None, verbose=False):
    M, N = X.shape
    avg_X = np.mean(X)
    W = np.random.rand(M, K) * np.sqrt(avg_X)
    H = np.random.rand(K, N) * np.sqrt(avg_X)

    prev_e = 0
    epsilon = eps
    for step in range(steps):
        WH = W @ H
        e = np.linalg.norm(X - WH, "fro")**2

        numerator_H = W.T @ X
        denominator_H = W.T @ W @ H

        epsilon = epsilon         
        H *= (numerator_H / (denominator_H + epsilon))

        numerator_W = X @ H.T
        denominator_W = W @ H @ H.T
        W *= (numerator_W / (denominator_W + epsilon))

        
        if verbose and step % 10 == 0:
            print(f'step:{step}, e:{e:.4f}')
            
        if step > 0:
            rel_change = abs(e - prev_e) / (prev_e + 1e-12)
            if rel_change < tol:   
                print(f'Converged at step {step}, e = {e:.4f}, rel_change={rel_change:.3e}')
                break
        prev_e = e

    return W, H, step+1

    
def stacked_nmf_pretrain(X, layer_ranks, steps_per_layer=200, tol=1e-3,  eps=1e-9, verbose=False):
    Y = X.copy()
    Ws = []
    H_final = None
    for i, K in enumerate(layer_ranks):
        if verbose:
            print(f"Pretraining layer {i+1}/{len(layer_ranks)}: factorizing matrix shape {Y.shape} -> rank {K}")
        W_i, H_i, step = nmf_mu(Y, K, steps = steps_per_layer, tol = tol, eps = eps, verbose = verbose)
        Ws.append(W_i)
        # for next layer, use H_i as data
        Y = H_i
        H_final = H_i  # final H will be last H_i
    return Ws, H_final, step

def reconstruct_from_stacked(Ws, H):
    """
    Compute full reconstruction WH = W1 W2 ... WL H.
    Ws: list of W_i, shapes must chain: W1(m,K1), W2(K1,K2), ...
    H: final H (K_L, n)
    """
    P = Ws[0]
    for W in Ws[1:]:
        P = P @ W
    return P @ H
    
def stackedNMF(X, layer_ranks=[10, 6], steps_per_layer = 100, tol = 1e-3, eps=1e-9):
    Ws, H_final, step = stacked_nmf_pretrain(X, layer_ranks, steps_per_layer=steps_per_layer, tol=tol,  eps=eps, verbose=False)
    PH = reconstruct_from_stacked(Ws, H_final)
    return H_final, PH, step

# Define Evaluation Indicators (RRE, ACC, NMI)

In [12]:
def relative_reconstruction_error(X, WH):
    """Compute the relative reconstruction error (RRE)."""
    return np.linalg.norm(X - WH, 'fro') / (np.linalg.norm(X, 'fro') + 1e-12)

def assign_cluster_label(X, Y):
    kmeans = KMeans(n_clusters=len(set(Y))).fit(X)
    Y_pred = np.zeros(Y.shape)
    for i in set(kmeans.labels_):
        ind = kmeans.labels_ == i
        Y_pred[ind] = Counter(Y[ind]).most_common(1)[0][0]
    return Y_pred
    
def evaluate_clustering(H, Y_true):
    Y_pred = assign_cluster_label(H.T, Y_true)
    acc = accuracy_score(Y_true, Y_pred)
    nmi = normalized_mutual_info_score(Y_true, Y_pred)
    print('Acc(NMI) = {:.4f} ({:.4f})'.format(acc, nmi))
    return acc, nmi

# Test on ORL

In [13]:
Data = 'ORL'

## L2-Norm Based NMF

In [69]:
Algo = 'L2'
Data = 'ORL'
dataset = 'data/ORL'
reduce = 3
X, Y = load_data(dataset, reduce=reduce)
K = len(set(Y))
seeds = [0,1,2,3,4]
pr_lst = [(0,0),(0.1,0.1),(0.2,0.2),(0.3,0.3),(0.4,0.4),(0.5,0.5)]
record_L2_ORL = []
steps = 10000
lr = 1e-3
tol = 1e-3


for seed in seeds:
    np.random.seed(seed)
    for p, r in pr_lst:
        # X_noisy, noise = salt_pepper(X, p, r)
        X_noisy, Y = load_data_salt_pepper(dataset, reduce, p, r)
        start = time.time()
        W, H ,WH, step = L2_nmf(X_noisy, K, lr, steps, tol, verbose=True)
        end = time.time()
        timeusing = end - start
        rre = relative_reconstruction_error(X_noisy, WH)
        acc, nmi = evaluate_clustering(H, Y)
        record_L2_ORL.append(((p,r),acc,nmi,rre,lr,tol,step,seed,timeusing))

step:0, e:27990.3098
step:10, e:120.8248
step:20, e:67.8195
step:30, e:63.0898
step:40, e:60.3754
step:50, e:57.9227
step:60, e:55.6929
step:70, e:53.6621
step:80, e:51.8106
step:90, e:50.1201
step:100, e:48.5749
step:110, e:47.1607
step:120, e:45.8649
step:130, e:44.6768
step:140, e:43.5866
step:150, e:42.5852
step:160, e:41.6643
step:170, e:40.8167
step:180, e:40.0360
step:190, e:39.3159
step:200, e:38.6509
step:210, e:38.0364
step:220, e:37.4678
step:230, e:36.9411
step:240, e:36.4525
step:250, e:35.9989
step:260, e:35.5770
step:270, e:35.1840
step:280, e:34.8172
Converged at step 284, e = 34.6773, rel_change=9.974e-04
Acc(NMI) = 0.2575 (0.4575)


D:\Software\Anaconda3\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1429: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=2.
  warnings.warn(


step:0, e:25652.0430
step:10, e:158.9074
step:20, e:101.9868
step:30, e:97.1986
step:40, e:94.7741
step:50, e:92.5843
step:60, e:90.5844
step:70, e:88.7555
step:80, e:87.0804
step:90, e:85.5442
step:100, e:84.1340
step:110, e:82.8381
step:120, e:81.6461
step:130, e:80.5491
step:140, e:79.5384
step:150, e:78.6069
step:160, e:77.7475
step:170, e:76.9541
Converged at step 170, e = 76.9541, rel_change=9.933e-04
Acc(NMI) = 0.1800 (0.3355)


D:\Software\Anaconda3\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1429: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=2.
  warnings.warn(


step:0, e:23676.8378
step:10, e:195.3109
step:20, e:133.5415
step:30, e:128.4356
step:40, e:126.3499
step:50, e:124.5005
step:60, e:122.8088
step:70, e:121.2585
step:80, e:119.8359
step:90, e:118.5288
Converged at step 99, e = 117.4422, rel_change=9.933e-04
Acc(NMI) = 0.1650 (0.2923)


...

D:\Software\Anaconda3\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1429: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=2.
  warnings.warn(


step:0, e:20235.8212
step:10, e:238.4147
step:20, e:170.2748
step:30, e:164.7840
Converged at step 34, e = 164.0746, rel_change=9.765e-04
Acc(NMI) = 0.1625 (0.3023)


D:\Software\Anaconda3\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1429: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=2.
  warnings.warn(


step:0, e:20586.1441
step:10, e:236.4743
step:20, e:169.8434
step:30, e:164.7485
Converged at step 32, e = 164.4128, rel_change=9.738e-04
Acc(NMI) = 0.1725 (0.3152)


D:\Software\Anaconda3\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1429: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=2.
  warnings.warn(


In [70]:
df_L2_ORL = pd.DataFrame(record_L2_ORL, columns=['(p,r)','acc','nmi','rre','lr','tol','step','seed', 'time'])
print(df_L2_ORL)
summary = df_L2_ORL.groupby(['(p,r)']).agg(
    RRE_mean=('rre', 'mean'),
    RRE_std=('rre', 'std'),
    ACC_mean=('acc', 'mean'),
    ACC_std=('acc', 'std'),
    NMI_mean=('nmi', 'mean'),
    NMI_std=('nmi', 'std')
).reset_index()
print(summary)
df_L2_ORL.to_excel(f'{Algo}_{Data}.xlsx', index=True)

         (p,r)     acc       nmi       rre     lr    tol  step  seed      time
0       (0, 0)  0.2575  0.457519  0.294437  0.001  0.001   285     0  1.481257
1   (0.1, 0.1)  0.1800  0.335515  0.438617  0.001  0.001   171     0  0.954528
2   (0.2, 0.2)  0.1650  0.292343  0.541854  0.001  0.001   100     0  0.586041
3   (0.3, 0.3)  0.1725  0.331080  0.610024  0.001  0.001    53     0  0.341824
4   (0.4, 0.4)  0.1600  0.327789  0.640747  0.001  0.001    35     0  0.214390
5   (0.5, 0.5)  0.1625  0.310667  0.642093  0.001  0.001    34     0  0.260028
6       (0, 0)  0.3050  0.479639  0.292698  0.001  0.001   288     1  1.541099
7   (0.1, 0.1)  0.1775  0.364984  0.439329  0.001  0.001   169     1  0.883424
8   (0.2, 0.2)  0.1800  0.333471  0.542887  0.001  0.001    98     1  0.555524
9   (0.3, 0.3)  0.1700  0.313990  0.609993  0.001  0.001    50     1  0.314520
10  (0.4, 0.4)  0.1825  0.328547  0.640469  0.001  0.001    35     1  0.234084
11  (0.5, 0.5)  0.1650  0.316366  0.641783  0.001  0

## Lee--Seung Multiplicative Update NMF (MU-L2)

In [71]:
Algo = 'MU'
Data = 'ORL'
dataset = 'data/ORL'
reduce = 3
X, Y = load_data(dataset, reduce=reduce)
K = len(set(Y))
seeds = [0,1,2,3,4]
pr_lst = [(0,0),(0.1,0.1),(0.2,0.2),(0.3,0.3),(0.4,0.4),(0.5,0.5)]
record_MU_ORL = []
steps = 10000
tol = 1e-3
epsilon=1e-9

for seed in seeds:
    np.random.seed(seed)
    for p, r in pr_lst:
        # X_noisy, noise = salt_pepper(X, p, r)
        X_noisy, Y = load_data_salt_pepper(dataset, reduce, p, r)
        start = time.time()
        W, H, WH, step = L2Lee_Seung(X_noisy, K, steps, tol, epsilon, verbose=True)
        end = time.time()
        timeusing = end - start
        rre = relative_reconstruction_error(X_noisy, WH)
        acc, nmi = evaluate_clustering(H, Y)
        record_MU_ORL.append(((p,r),acc,nmi,rre,tol,step,seed, timeusing))

step:0, e:27990.3098
step:10, e:32.9568
step:20, e:29.4600
step:30, e:23.3307
step:40, e:18.9276
step:50, e:16.0972
step:60, e:14.2564
step:70, e:12.9469
step:80, e:11.9709
step:90, e:11.2182
step:100, e:10.6164
step:110, e:10.1214
step:120, e:9.7075
step:130, e:9.3581
step:140, e:9.0609
step:150, e:8.8064
step:160, e:8.5871
step:170, e:8.3973
step:180, e:8.2320
step:190, e:8.0875
step:200, e:7.9607
step:210, e:7.8489
step:220, e:7.7503
step:230, e:7.6629
Converged at step 238, e = 7.6002, rel_change=9.897e-04
Acc(NMI) = 0.7025 (0.8283)


D:\Software\Anaconda3\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1429: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=2.
  warnings.warn(


step:0, e:25652.0430
step:10, e:69.6239
step:20, e:66.5013
step:30, e:60.7676
step:40, e:56.2927
step:50, e:53.2752
step:60, e:51.3006
step:70, e:49.8994
step:80, e:48.8517
step:90, e:48.0458
step:100, e:47.4128
Converged at step 109, e = 46.9522, rel_change=9.990e-04


D:\Software\Anaconda3\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1429: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=2.
  warnings.warn(


Acc(NMI) = 0.6150 (0.7710)
step:0, e:23676.8378
step:10, e:104.8333
step:20, e:101.3426
step:30, e:96.1211
step:40, e:92.0050
step:50, e:89.0936
step:60, e:87.0749
step:70, e:85.6286
step:80, e:84.5545
Converged at step 85, e = 84.1174, rel_change=9.854e-04
Acc(NMI) = 0.4200 (0.6034)


...

D:\Software\Anaconda3\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1429: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=2.
  warnings.warn(


step:0, e:20235.8212
step:10, e:148.1977
step:20, e:144.5477
step:30, e:140.0241
step:40, e:136.1003
step:50, e:133.3622
step:60, e:131.4525
Converged at step 67, e = 130.4471, rel_change=9.987e-04
Acc(NMI) = 0.2125 (0.3761)


D:\Software\Anaconda3\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1429: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=2.
  warnings.warn(


step:0, e:20586.1441
step:10, e:150.5844
step:20, e:147.0702
step:30, e:142.9982
step:40, e:139.4409
step:50, e:136.8541
step:60, e:135.0044
Converged at step 66, e = 134.1441, rel_change=9.887e-04
Acc(NMI) = 0.1850 (0.3517)


D:\Software\Anaconda3\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1429: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=2.
  warnings.warn(


In [72]:
df_MU_ORL = pd.DataFrame(record_MU_ORL, columns=['(p,r)','acc','nmi','rre','tol','step','seed','time'])
print(df_MU_ORL)
summary = df_MU_ORL.groupby(['(p,r)']).agg(
    RRE_mean=('rre', 'mean'),
    RRE_std=('rre', 'std'),
    ACC_mean=('acc', 'mean'),
    ACC_std=('acc', 'std'),
    NMI_mean=('nmi', 'mean'),
    NMI_std=('nmi', 'std')
).reset_index()
print(summary)
df_MU_ORL.to_excel(f'{Algo}_{Data}.xlsx', index=True)

         (p,r)     acc       nmi       rre    tol  step  seed      time
0       (0, 0)  0.7025  0.828253  0.137842  0.001   239     0  1.246517
1   (0.1, 0.1)  0.6150  0.770987  0.342608  0.001   110     0  0.624735
2   (0.2, 0.2)  0.4200  0.603352  0.458578  0.001    86     0  0.477152
3   (0.3, 0.3)  0.2925  0.471869  0.532614  0.001    75     0  0.426628
4   (0.4, 0.4)  0.2325  0.414753  0.571609  0.001    68     0  0.412698
5   (0.5, 0.5)  0.1925  0.357068  0.579991  0.001    65     0  0.403633
6       (0, 0)  0.7050  0.832489  0.138128  0.001   235     1  1.206605
7   (0.1, 0.1)  0.6100  0.758731  0.342026  0.001   111     1  0.617091
8   (0.2, 0.2)  0.4000  0.587405  0.459666  0.001    83     1  0.424754
9   (0.3, 0.3)  0.2950  0.474355  0.532501  0.001    73     1  0.435355
10  (0.4, 0.4)  0.2275  0.401014  0.571511  0.001    68     1  0.349675
11  (0.5, 0.5)  0.1900  0.335312  0.579764  0.001    66     1  0.379247
12      (0, 0)  0.6975  0.821575  0.139249  0.001   233     2  1

## L1-Norm Regularized Robust NMF

In [14]:
Algo = 'L1'
Data = 'ORL'
dataset = 'data/ORL'
reduce = 3
X, Y = load_data(dataset, reduce=reduce)
eps=1e-8
rank = 40
steps = 10000
lambdas = [(0.0,0.0),(0.01,0.005)]
kmeans_init = 5
verbose_nmf = False
seeds = [0,1,2,3,4]
pr_lst = [(0,0),(0.1,0.1),(0.2,0.2),(0.3,0.3),(0.4,0.4),(0.5,0.5)]
tol = 1e-3
record_L1_ORL = []

for seed in seeds:
    np.random.seed(seed)
    for lambda_h, lambda_w in lambdas:
        for p, r in pr_lst:
            # X_noisy, noise = salt_pepper(X, p, r)
            X_noisy, Y = load_data_salt_pepper(dataset, reduce, p, r)
            start = time.time()
            W, H, WH, step = l1_nmf_corrected(X, rank=rank, steps=steps, tol=tol, lambda_h=lambda_h, lambda_w=lambda_w, eps=eps)
            end = time.time()
            timeusing = end - start
            rre = relative_reconstruction_error(X_noisy, WH)
            acc, nmi = evaluate_clustering(H, Y)
            record_L1_ORL.append(((p,r), (lambda_h,lambda_w), acc,nmi,rre,step, seed, timeusing))

C:\Users\lenovo\AppData\Local\Temp\ipykernel_1840\282225574.py:35: RuntimeWarning: invalid value encountered in scalar divide
  if abs(E_sq - prev_E) / (prev_E + 1e-12) < tol:
D:\Software\Anaconda3\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1429: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=2.
  warnings.warn(


Acc(NMI) = 0.6800 (0.8161)


C:\Users\lenovo\AppData\Local\Temp\ipykernel_1840\282225574.py:35: RuntimeWarning: invalid value encountered in scalar divide
  if abs(E_sq - prev_E) / (prev_E + 1e-12) < tol:
D:\Software\Anaconda3\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1429: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=2.
  warnings.warn(


Acc(NMI) = 0.6850 (0.8208)


C:\Users\lenovo\AppData\Local\Temp\ipykernel_1840\282225574.py:35: RuntimeWarning: invalid value encountered in scalar divide
  if abs(E_sq - prev_E) / (prev_E + 1e-12) < tol:
D:\Software\Anaconda3\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1429: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=2.
  warnings.warn(


...

C:\Users\lenovo\AppData\Local\Temp\ipykernel_1840\282225574.py:35: RuntimeWarning: invalid value encountered in scalar divide
  if abs(E_sq - prev_E) / (prev_E + 1e-12) < tol:
D:\Software\Anaconda3\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1429: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=2.
  warnings.warn(


Acc(NMI) = 0.7025 (0.8337)


C:\Users\lenovo\AppData\Local\Temp\ipykernel_1840\282225574.py:35: RuntimeWarning: invalid value encountered in scalar divide
  if abs(E_sq - prev_E) / (prev_E + 1e-12) < tol:


Acc(NMI) = 0.7375 (0.8557)


D:\Software\Anaconda3\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1429: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=2.
  warnings.warn(


In [15]:
df_L1_ORL = pd.DataFrame(record_L1_ORL, columns=['(p,r)', '(lambda_h,lambda_w)','acc','nmi','rre','step','seed','time'])
print(df_L1_ORL)
summary = df_L1_ORL.groupby(['(p,r)']).agg(
    RRE_mean=('rre', 'mean'),
    RRE_std=('rre', 'std'),
    ACC_mean=('acc', 'mean'),
    ACC_std=('acc', 'std'),
    NMI_mean=('nmi', 'mean'),
    NMI_std=('nmi', 'std')
).reset_index()
print(summary)
df_L1_ORL.to_excel(f'{Algo}_{Data}.xlsx', index=True)

         (p,r) (lambda_h,lambda_w)     acc       nmi       rre  step  seed  \
0       (0, 0)          (0.0, 0.0)  0.6800  0.816082  0.139027     8     0   
1   (0.1, 0.1)          (0.0, 0.0)  0.6850  0.820772  0.354142     8     0   
2   (0.2, 0.2)          (0.0, 0.0)  0.6950  0.837705  0.489061     8     0   
3   (0.3, 0.3)          (0.0, 0.0)  0.7250  0.840985  0.583191     8     0   
4   (0.4, 0.4)          (0.0, 0.0)  0.6700  0.801858  0.640879     8     0   
5   (0.5, 0.5)          (0.0, 0.0)  0.7125  0.839191  0.664392     8     0   
6       (0, 0)       (0.01, 0.005)  0.7125  0.831069  0.150310     7     0   
7   (0.1, 0.1)       (0.01, 0.005)  0.6525  0.809277  0.357154     7     0   
8   (0.2, 0.2)       (0.01, 0.005)  0.7375  0.863079  0.489907     7     0   
9   (0.3, 0.3)       (0.01, 0.005)  0.7275  0.855431  0.583907     7     0   
10  (0.4, 0.4)       (0.01, 0.005)  0.6850  0.829755  0.640463     7     0   
11  (0.5, 0.5)       (0.01, 0.005)  0.6700  0.815357  0.664093  

## Hypersurface Cost-Based NMF (HCNMF)

In [41]:
Algo = 'HC'
Data = 'ORL'
dataset = 'data/ORL'
reduce = 3
X, Y = load_data(dataset, reduce=reduce)
rank = 30
steps = 10000
deltas = [0.05, 0.1, 0.2]
eps = 1e-8
verbose = False
seeds = [0,1,2,3,4]
pr_lst = [(0,0),(0.1,0.1),(0.2,0.2),(0.3,0.3),(0.4,0.4),(0.5,0.5)]
record_HC_ORL = []
tol = 1e-3

for seed in seeds:
    np.random.seed(seed)
    for delta in deltas:
        for p, r in pr_lst:
            # X_noisy, noise = salt_pepper(X, p, r)
            X_noisy, Y = load_data_salt_pepper(dataset, reduce, p, r)
            start = time.time()
            W, H, WH, step = hypersurface_nmf(X_noisy, rank, steps, tol, delta, eps, verbose)
            end = time.time()
            timeusing = end - start
            rre = relative_reconstruction_error(X_noisy, WH)
            acc, nmi = evaluate_clustering(H, Y)
            record_HC_ORL.append(((p,r),delta,acc,nmi,rre,step, seed, timeusing))

D:\Software\Anaconda3\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1429: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=2.
  warnings.warn(


Acc(NMI) = 0.6950 (0.8332)


D:\Software\Anaconda3\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1429: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=2.
  warnings.warn(


Acc(NMI) = 0.6250 (0.8094)


D:\Software\Anaconda3\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1429: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=2.
  warnings.warn(


...

D:\Software\Anaconda3\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1429: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=2.
  warnings.warn(


Acc(NMI) = 0.2775 (0.4559)


D:\Software\Anaconda3\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1429: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=2.
  warnings.warn(


Acc(NMI) = 0.2150 (0.3981)
Acc(NMI) = 0.2000 (0.3599)


D:\Software\Anaconda3\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1429: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=2.
  warnings.warn(


In [44]:
df_HC_ORL = pd.DataFrame(record_HC_ORL, columns=['(p,r)','delta','acc','nmi','rre','step','seed','time'])
print(df_HC_ORL)
summary = df_HC_ORL.groupby(['(p,r)','delta']).agg(
    RRE_mean=('rre', 'mean'),
    RRE_std=('rre', 'std'),
    ACC_mean=('acc', 'mean'),
    ACC_std=('acc', 'std'),
    NMI_mean=('nmi', 'mean'),
    NMI_std=('nmi', 'std')
).reset_index()
print(summary)
df_HC_ORL.to_excel(f'{Algo}_{Data}.xlsx', index=True)

         (p,r)  delta     acc       nmi       rre  step  seed      time
0       (0, 0)   0.05  0.6950  0.833225  0.150220   201     0  8.158253
1   (0.1, 0.1)   0.05  0.6250  0.809417  0.350390   105     0  4.443236
2   (0.2, 0.2)   0.05  0.4325  0.627173  0.468609    77     0  2.856597
3   (0.3, 0.3)   0.05  0.3300  0.509313  0.543202    67     0  2.792086
4   (0.4, 0.4)   0.05  0.2350  0.416473  0.581839    61     0  2.453914
..         ...    ...     ...       ...       ...   ...   ...       ...
85  (0.1, 0.1)   0.20  0.5925  0.747222  0.350215   104     4  3.726791
86  (0.2, 0.2)   0.20  0.4550  0.625737  0.466958    79     4  2.779315
87  (0.3, 0.3)   0.20  0.2775  0.455925  0.543176    65     4  2.212155
88  (0.4, 0.4)   0.20  0.2150  0.398075  0.581242    59     4  2.165703
89  (0.5, 0.5)   0.20  0.2000  0.359893  0.589682    55     4  1.878978

[90 rows x 8 columns]
         (p,r)  delta  RRE_mean   RRE_std  ACC_mean   ACC_std  NMI_mean  \
0       (0, 0)   0.05  0.150044  0.000

## Stacked NMF

In [77]:
Algo = 'StackedNMF'
Data = 'ORL'
dataset = 'data/ORL'
reduce = 3
X, Y = load_data(dataset, reduce=reduce)
seeds = [0,1,2,3,4]
pr_lst = [(0,0),(0.1,0.1),(0.2,0.2),(0.3,0.3),(0.4,0.4),(0.5,0.5)]
record_SNMF_ORL = []
layer_ranks=[10, 6]
steps_per_layer = 10000
tol = 1e-3
eps = 1e-9

for seed in seeds:
    np.random.seed(seed)
    for p, r in pr_lst:
        # X_noisy, noise = salt_pepper(X, p, r)
        X_noisy, Y = load_data_salt_pepper(dataset, reduce, p, r)
        start = time.time()
        H_final, PH, step = stackedNMF(X_noisy, layer_ranks=layer_ranks, steps_per_layer = steps_per_layer, tol = tol, eps=eps)
        end = time.time()
        timeusing = end - start
        rre = relative_reconstruction_error(X_noisy, PH)
        acc, nmi = evaluate_clustering(H_final, Y)
        record_SNMF_ORL.append(((p,r),acc,nmi,rre,lr,tol,step,seed,timeusing))

Converged at step 136, e = 14.4531, rel_change=9.744e-04
Converged at step 61, e = 0.7623, rel_change=9.733e-04
Acc(NMI) = 0.5325 (0.7157)


D:\Software\Anaconda3\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1429: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=2.
  warnings.warn(


Converged at step 93, e = 55.2911, rel_change=9.793e-04
Converged at step 132, e = 0.5305, rel_change=9.625e-04
Acc(NMI) = 0.4975 (0.7017)


D:\Software\Anaconda3\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1429: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=2.
  warnings.warn(


Converged at step 69, e = 95.5693, rel_change=9.968e-04
Converged at step 64, e = 0.4888, rel_change=9.757e-04
Acc(NMI) = 0.3950 (0.5924)


...

D:\Software\Anaconda3\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1429: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=2.
  warnings.warn(


Converged at step 18, e = 148.6512, rel_change=9.951e-04
Converged at step 61, e = 0.3476, rel_change=9.745e-04
Acc(NMI) = 0.1800 (0.3157)


D:\Software\Anaconda3\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1429: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=2.
  warnings.warn(


Converged at step 16, e = 151.8212, rel_change=9.588e-04
Converged at step 85, e = 0.3594, rel_change=9.844e-04
Acc(NMI) = 0.1700 (0.3273)


D:\Software\Anaconda3\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1429: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=2.
  warnings.warn(


In [78]:
df_SNMF_ORL = pd.DataFrame(record_SNMF_ORL, columns=['(p,r)','acc','nmi','rre','lr','tol','step','seed','time'])
print(df_SNMF_ORL)
summary = df_SNMF_ORL.groupby(['(p,r)']).agg(
    RRE_mean=('rre', 'mean'),
    RRE_std=('rre', 'std'),
    ACC_mean=('acc', 'mean'),
    ACC_std=('acc', 'std'),
    NMI_mean=('nmi', 'mean'),
    NMI_std=('nmi', 'std')
).reset_index()
print(summary)
df_SNMF_ORL.to_excel(f'{Algo}_{Data}.xlsx', index=True)

         (p,r)     acc       nmi       rre     lr    tol  step  seed      time
0       (0, 0)  0.5325  0.715674  0.219500  0.001  0.001    62     0  0.584959
1   (0.1, 0.1)  0.4975  0.701678  0.381928  0.001  0.001   133     0  0.473217
2   (0.2, 0.2)  0.3950  0.592424  0.493999  0.001  0.001    65     0  0.384119
3   (0.3, 0.3)  0.2675  0.470878  0.568273  0.001  0.001    72     0  0.294048
4   (0.4, 0.4)  0.1825  0.333489  0.611962  0.001  0.001    88     0  0.095504
5   (0.5, 0.5)  0.1675  0.322474  0.616782  0.001  0.001    82     0  0.083764
6       (0, 0)  0.5525  0.752816  0.224038  0.001  0.001    71     1  0.678495
7   (0.1, 0.1)  0.4975  0.698349  0.382984  0.001  0.001    78     1  0.494241
8   (0.2, 0.2)  0.3550  0.566730  0.493525  0.001  0.001    79     1  0.354545
9   (0.3, 0.3)  0.2525  0.447318  0.568983  0.001  0.001    73     1  0.220610
10  (0.4, 0.4)  0.1600  0.327497  0.611817  0.001  0.001    83     1  0.113051
11  (0.5, 0.5)  0.1500  0.290090  0.617169  0.001  0

# Test on CroppedYaleB

In [79]:
Data = 'YaleB'

## L2-Norm Based NMF

In [80]:
Algo = 'L2'
Data = 'YaleB'
dataset = 'data/CroppedYaleB'
reduce = 3
X, Y = load_data(dataset, reduce=reduce)
K = len(set(Y))
seeds = [0,1,2,3,4]
pr_lst = [(0,0),(0.1,0.1),(0.2,0.2),(0.3,0.3),(0.4,0.4),(0.5,0.5)]
record_L2_Yale = []
steps = 10000
lrs = [1e-3]
tol = 1e-3

for seed in seeds:
    np.random.seed(seed)
    for lr in lrs:
        for p, r in pr_lst:
            # X_noisy, noise = salt_pepper(X, p, r)
            X_noisy, Y = load_data_salt_pepper(dataset, reduce, p, r)
            start = time.time()
            W, H ,WH, step = L2_nmf(X_noisy, K, lr, steps, tol, verbose=True)
            end = time.time()
            timeusing = end - start
            rre = relative_reconstruction_error(X_noisy, WH)
            acc, nmi = evaluate_clustering(H, Y)
            record_L2_Yale.append(((p,r),acc,nmi,rre,lr,tol,step,seed,timeusing))

step:0, e:101811.4300
step:10, e:1091.6720
step:20, e:1017.0635
step:30, e:996.6994
step:40, e:983.5426
step:50, e:972.4709
step:60, e:961.2257
step:70, e:947.5288
step:80, e:928.5840
step:90, e:901.1311
step:100, e:862.1889
step:110, e:811.0626
step:120, e:751.4299
step:130, e:690.9679
step:140, e:637.5064
step:150, e:594.5329
step:160, e:561.3444
step:170, e:535.4415
step:180, e:514.3805
step:190, e:496.4440
step:200, e:480.6418
step:210, e:466.4509
step:220, e:453.5668
step:230, e:441.7994
step:240, e:430.9524
step:250, e:420.8692
step:260, e:411.4289
step:270, e:402.5340
step:280, e:394.1072
step:290, e:386.0924
step:300, e:378.4843
step:310, e:371.2653
step:320, e:364.4225
step:330, e:357.9467
step:340, e:351.8200
step:350, e:346.0323
step:360, e:340.5737
step:370, e:335.4345
step:380, e:330.5984
step:390, e:326.0456
step:400, e:321.7580
step:410, e:317.7202
step:420, e:313.9122
step:430, e:310.3155
step:440, e:306.9124
step:450, e:303.6883
Converged at step 459, e = 300.9283, rel

D:\Software\Anaconda3\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1429: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=10.
  warnings.warn(


step:0, e:88227.1129
step:10, e:1281.3523
step:20, e:1193.7555
step:30, e:1174.1839
Converged at step 35, e = 1168.0103, rel_change=9.790e-04
Acc(NMI) = 0.0746 (0.0592)


D:\Software\Anaconda3\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1429: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=10.
  warnings.warn(


step:0, e:79456.7039
step:10, e:1429.9660
step:20, e:1334.9421
Converged at step 29, e = 1318.0994, rel_change=9.765e-04
Acc(NMI) = 0.0775 (0.0724)


...

D:\Software\Anaconda3\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1429: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=10.
  warnings.warn(


step:0, e:77921.3624
step:10, e:1489.3346
step:20, e:1398.8971
Converged at step 25, e = 1390.1006, rel_change=9.494e-04
Acc(NMI) = 0.0775 (0.0621)


D:\Software\Anaconda3\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1429: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=10.
  warnings.warn(


step:0, e:84656.5748
step:10, e:1402.1024
step:20, e:1321.6316
Converged at step 24, e = 1315.1256, rel_change=9.895e-04
Acc(NMI) = 0.0771 (0.0681)


D:\Software\Anaconda3\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1429: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=10.
  warnings.warn(


In [81]:
df_L2_Yale = pd.DataFrame(record_L2_Yale, columns=['(p,r)','acc','nmi','rre','lr','tol','step','seed','time'])
print(df_L2_Yale)
summary = df_L2_Yale.groupby(['(p,r)']).agg(
    RRE_mean=('rre', 'mean'),
    RRE_std=('rre', 'std'),
    ACC_mean=('acc', 'mean'),
    ACC_std=('acc', 'std'),
    NMI_mean=('nmi', 'mean'),
    NMI_std=('nmi', 'std')
).reset_index()
print(summary)
df_L2_Yale.to_excel(f'{Algo}_{Data}.xlsx', index=True)

         (p,r)       acc       nmi       rre     lr    tol  step  seed  \
0       (0, 0)  0.069594  0.036998  0.353072  0.001  0.001   460     0   
1   (0.1, 0.1)  0.074565  0.059213  0.695592  0.001  0.001    36     0   
2   (0.2, 0.2)  0.077465  0.072377  0.738934  0.001  0.001    30     0   
3   (0.3, 0.3)  0.074979  0.060988  0.760033  0.001  0.001    27     0   
4   (0.4, 0.4)  0.076222  0.066373  0.758891  0.001  0.001    26     0   
5   (0.5, 0.5)  0.078708  0.067490  0.737993  0.001  0.001    25     0   
6       (0, 0)  0.067937  0.037178  0.353642  0.001  0.001   465     1   
7   (0.1, 0.1)  0.072080  0.056955  0.695035  0.001  0.001    37     1   
8   (0.2, 0.2)  0.082436  0.070626  0.738745  0.001  0.001    30     1   
9   (0.3, 0.3)  0.078293  0.060236  0.759473  0.001  0.001    28     1   
10  (0.4, 0.4)  0.080779  0.068633  0.758938  0.001  0.001    26     1   
11  (0.5, 0.5)  0.077465  0.063036  0.738039  0.001  0.001    25     1   
12      (0, 0)  0.066694  0.036382  0.

## Lee--Seung Multiplicative Update NMF (MU-L2)

In [82]:
Algo = 'MU'
Data = 'YaleB'
dataset = 'data/CroppedYaleB'
reduce = 3
X, Y = load_data(dataset, reduce=reduce)
K = len(set(Y))
seeds = [0,1,2,3,4]
pr_lst = [(0,0),(0.1,0.1),(0.2,0.2),(0.3,0.3),(0.4,0.4),(0.5,0.5)]
record_MU_Yale = []
steps = 10000
tol = 1e-3
epsilon=1e-9

for seed in seeds:
    np.random.seed(seed)
    for p, r in pr_lst:
        # X_noisy, noise = salt_pepper(X, p, r)
        X_noisy, Y = load_data_salt_pepper(dataset, reduce, p, r)
        start = time.time()
        W, H, WH, step = L2Lee_Seung(X_noisy, K, steps, tol, epsilon, verbose=True)
        end = time.time()
        timeusing = end - start
        rre = relative_reconstruction_error(X_noisy, WH)
        acc, nmi = evaluate_clustering(H, Y)
        record_MU_Yale.append(((p,r),acc,nmi,rre,tol,step,seed,timeusing))

step:0, e:101811.4300
step:10, e:581.8430
step:20, e:335.0237
step:30, e:270.8006
step:40, e:239.8093
step:50, e:219.9102
step:60, e:205.2410
step:70, e:193.5952
step:80, e:184.3717
step:90, e:177.3380
step:100, e:172.0629
step:110, e:168.0988
step:120, e:165.0522
step:130, e:162.6597
step:140, e:160.7750
Converged at step 143, e = 160.2855, rel_change=9.960e-04
Acc(NMI) = 0.1624 (0.2657)


D:\Software\Anaconda3\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1429: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=10.
  warnings.warn(


step:0, e:88227.1129
step:10, e:851.2188
step:20, e:680.5429
step:30, e:623.5162
step:40, e:601.9416
step:50, e:585.8857
step:60, e:573.9052
step:70, e:564.6042
step:80, e:557.1729
Converged at step 88, e = 552.3641, rel_change=9.970e-04
Acc(NMI) = 0.1417 (0.2021)


D:\Software\Anaconda3\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1429: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=10.
  warnings.warn(


step:0, e:79456.7039
Converged at step 2, e = 1270.3754, rel_change=8.097e-04


...

D:\Software\Anaconda3\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1429: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=10.
  warnings.warn(


Acc(NMI) = 0.0779 (0.0706)
step:0, e:77921.3624
Converged at step 2, e = 1345.5945, rel_change=5.087e-04


D:\Software\Anaconda3\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1429: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=10.
  warnings.warn(


Acc(NMI) = 0.0766 (0.0633)
step:0, e:84656.5748
Converged at step 2, e = 1268.9052, rel_change=4.973e-04
Acc(NMI) = 0.0804 (0.0676)


D:\Software\Anaconda3\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1429: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=10.
  warnings.warn(


In [83]:
df_MU_Yale = pd.DataFrame(record_MU_Yale, columns=['(p,r)','acc','nmi','rre','tol','step','seed','time'])
print(df_MU_Yale)
summary = df_MU_Yale.groupby(['(p,r)']).agg(
    RRE_mean=('rre', 'mean'),
    RRE_std=('rre', 'std'),
    ACC_mean=('acc', 'mean'),
    ACC_std=('acc', 'std'),
    NMI_mean=('nmi', 'mean'),
    NMI_std=('nmi', 'std')
).reset_index()
print(summary)
df_MU_Yale.to_excel(f'{Algo}_{Data}.xlsx', index=True)

         (p,r)       acc       nmi       rre    tol  step  seed       time
0       (0, 0)  0.162386  0.265696  0.257679  0.001   144     0  12.274826
1   (0.1, 0.1)  0.141674  0.202107  0.478348  0.001    89     0   7.628738
2   (0.2, 0.2)  0.074979  0.062532  0.725433  0.001     3     0   0.256994
3   (0.3, 0.3)  0.077051  0.065703  0.747295  0.001     3     0   0.265795
4   (0.4, 0.4)  0.075808  0.066915  0.746557  0.001     3     0   0.258014
5   (0.5, 0.5)  0.080365  0.071123  0.724958  0.001     3     0   0.267761
6       (0, 0)  0.152444  0.249392  0.257877  0.001   142     1  13.848859
7   (0.1, 0.1)  0.135046  0.202497  0.475914  0.001    89     1   8.237255
8   (0.2, 0.2)  0.077051  0.062117  0.725345  0.001     3     1   0.266633
9   (0.3, 0.3)  0.074979  0.068729  0.747192  0.001     3     1   0.265741
10  (0.4, 0.4)  0.076222  0.057950  0.746628  0.001     3     1   0.267510
11  (0.5, 0.5)  0.076636  0.062910  0.724969  0.001     3     1   0.273579
12      (0, 0)  0.153687 

## L1-Norm Regularized Robust NMF

In [16]:
Algo = 'L1'
Data = 'YaleB'
dataset = 'data/CroppedYaleB'
reduce = 3
X, Y = load_data(dataset, reduce=reduce)
eps=1e-8
rank = 40
steps = 10000
lambdas = [(0.0,0.0),(0.01,0.005)]
kmeans_init = 5
verbose_nmf = False
seeds = [0,1,2,3,4]
pr_lst = [(0,0),(0.1,0.1),(0.2,0.2),(0.3,0.3),(0.4,0.4),(0.5,0.5)]
record_L1_Yale = []
tol = 1e-3

for seed in seeds:
    np.random.seed(seed)
    for lambda_h, lambda_w in lambdas:
        for p, r in pr_lst:
            # X_noisy, noise = salt_pepper(X, p, r)
            X_noisy, Y = load_data_salt_pepper(dataset, reduce, p, r)
            start = time.time()
            W, H, WH, step = l1_nmf_corrected(X, rank=rank, steps=steps, tol=tol, lambda_h=lambda_h, lambda_w=lambda_w, eps=eps)
            end = time.time()
            timeusing = end - start
            rre = relative_reconstruction_error(X_noisy, WH)
            acc, nmi = evaluate_clustering(H, Y)
            record_L1_Yale.append(((p,r),(lambda_h,lambda_w),acc,nmi,rre,step,seed,timeusing))

C:\Users\lenovo\AppData\Local\Temp\ipykernel_1840\282225574.py:35: RuntimeWarning: invalid value encountered in scalar divide
  if abs(E_sq - prev_E) / (prev_E + 1e-12) < tol:
D:\Software\Anaconda3\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1429: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=10.
  warnings.warn(


Acc(NMI) = 0.1549 (0.2344)


C:\Users\lenovo\AppData\Local\Temp\ipykernel_1840\282225574.py:35: RuntimeWarning: invalid value encountered in scalar divide
  if abs(E_sq - prev_E) / (prev_E + 1e-12) < tol:
D:\Software\Anaconda3\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1429: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=10.
  warnings.warn(


Acc(NMI) = 0.1628 (0.2597)


C:\Users\lenovo\AppData\Local\Temp\ipykernel_1840\282225574.py:35: RuntimeWarning: invalid value encountered in scalar divide
  if abs(E_sq - prev_E) / (prev_E + 1e-12) < tol:
D:\Software\Anaconda3\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1429: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=10.
  warnings.warn(


...

C:\Users\lenovo\AppData\Local\Temp\ipykernel_1840\282225574.py:35: RuntimeWarning: invalid value encountered in scalar divide
  if abs(E_sq - prev_E) / (prev_E + 1e-12) < tol:
D:\Software\Anaconda3\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1429: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=10.
  warnings.warn(


Acc(NMI) = 0.1848 (0.2812)


C:\Users\lenovo\AppData\Local\Temp\ipykernel_1840\282225574.py:35: RuntimeWarning: invalid value encountered in scalar divide
  if abs(E_sq - prev_E) / (prev_E + 1e-12) < tol:


Acc(NMI) = 0.1777 (0.2689)


D:\Software\Anaconda3\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1429: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=10.
  warnings.warn(


In [17]:
df_L1_Yale = pd.DataFrame(record_L1_Yale, columns=['(p,r)','(lambda_h,lambda_w)','acc','nmi','rre','step','seed','time'])
print(df_L1_Yale)
summary = df_L1_Yale.groupby(['(p,r)']).agg(
    RRE_mean=('rre', 'mean'),
    RRE_std=('rre', 'std'),
    ACC_mean=('acc', 'mean'),
    ACC_std=('acc', 'std'),
    NMI_mean=('nmi', 'mean'),
    NMI_std=('nmi', 'std')
).reset_index()
print(summary)
df_L1_Yale.to_excel(f'{Algo}_{Data}.xlsx', index=True)

         (p,r) (lambda_h,lambda_w)       acc       nmi       rre  step  seed  \
0       (0, 0)          (0.0, 0.0)  0.154930  0.234400  0.261749     5     0   
1   (0.1, 0.1)          (0.0, 0.0)  0.162800  0.259703  0.482694     5     0   
2   (0.2, 0.2)          (0.0, 0.0)  0.157415  0.238696  0.645564     5     0   
3   (0.3, 0.3)          (0.0, 0.0)  0.148302  0.239827  0.747685     5     0   
4   (0.4, 0.4)          (0.0, 0.0)  0.163215  0.241310  0.804215     5     0   
5   (0.5, 0.5)          (0.0, 0.0)  0.170671  0.264484  0.826153     5     0   
6       (0, 0)       (0.01, 0.005)  0.193869  0.274555  0.262421     5     0   
7   (0.1, 0.1)       (0.01, 0.005)  0.199669  0.284270  0.483122     5     0   
8   (0.2, 0.2)       (0.01, 0.005)  0.173571  0.260163  0.645174     5     0   
9   (0.3, 0.3)       (0.01, 0.005)  0.200083  0.290216  0.748197     5     0   
10  (0.4, 0.4)       (0.01, 0.005)  0.216239  0.286825  0.805111     5     0   
11  (0.5, 0.5)       (0.01, 0.005)  0.17

## Hypersurface Cost-Based NMF (HCNMF)

In [45]:
Algo = 'HC'
Data = 'YaleB'
dataset = 'data/CroppedYaleB'
reduce = 3
X, Y = load_data(dataset, reduce=reduce)
rank = 30
steps = 10000
deltas = [0.1,0.2,0.5]
eps = 1e-8
verbose = False
seeds = [0,1,2,3,4]
pr_lst = [(0,0),(0.1,0.1),(0.2,0.2),(0.3,0.3),(0.4,0.4),(0.5,0.5)]
record_HC_Yale = []
tol = 1e-3

for seed in seeds:
    np.random.seed(seed)
    for delta in deltas:
        for p, r in pr_lst:
            # X_noisy, noise = salt_pepper(X, p, r)
            X_noisy, Y = load_data_salt_pepper(dataset, reduce, p, r)
            start = time.time()
            W, H, WH, step = hypersurface_nmf(X_noisy, rank, steps, tol, delta, eps, verbose)
            end = time.time()
            timeusing = end - start
            rre = relative_reconstruction_error(X_noisy, WH)
            acc, nmi = evaluate_clustering(H, Y)
            record_HC_Yale.append(((p,r),delta,acc,nmi,rre, step, seed, timeusing))

D:\Software\Anaconda3\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1429: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=10.
  warnings.warn(


Acc(NMI) = 0.1355 (0.2321)


D:\Software\Anaconda3\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1429: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=10.
  warnings.warn(


Acc(NMI) = 0.1355 (0.1983)


D:\Software\Anaconda3\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1429: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=10.
  warnings.warn(


...

D:\Software\Anaconda3\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1429: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=10.
  warnings.warn(


Acc(NMI) = 0.0742 (0.0627)


D:\Software\Anaconda3\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1429: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=10.
  warnings.warn(


Acc(NMI) = 0.0742 (0.0601)
Acc(NMI) = 0.0742 (0.0616)


D:\Software\Anaconda3\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1429: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=10.
  warnings.warn(


In [47]:
df_HC_Yale = pd.DataFrame(record_HC_Yale, columns=['(p,r)','delta','acc','nmi','rre','step','seed','time'])
print(df_HC_Yale)
summary = df_HC_Yale.groupby(['(p,r)','delta']).agg(
    RRE_mean=('rre', 'mean'),
    RRE_std=('rre', 'std'),
    ACC_mean=('acc', 'mean'),
    ACC_std=('acc', 'std'),
    NMI_mean=('nmi', 'mean'),
    NMI_std=('nmi', 'std')
).reset_index()
print(summary)
df_HC_Yale.to_excel(f'{Algo}_{Data}.xlsx', index=True)

         (p,r)  delta       acc       nmi       rre  step  seed       time
0       (0, 0)    0.1  0.135460  0.232057  0.273604   129     0  77.480723
1   (0.1, 0.1)    0.1  0.135460  0.198274  0.481477    85     0  52.030591
2   (0.2, 0.2)    0.1  0.095278  0.112425  0.620534    54     0  32.791454
3   (0.3, 0.3)    0.1  0.080779  0.066166  0.692525    41     0  24.278158
4   (0.4, 0.4)    0.1  0.073322  0.059158  0.747842     3     0   1.824203
..         ...    ...       ...       ...       ...   ...   ...        ...
85  (0.1, 0.1)    0.5  0.125104  0.188803  0.481680    80     4  49.991976
86  (0.2, 0.2)    0.5  0.091549  0.093508  0.620890    51     4  31.168733
87  (0.3, 0.3)    0.5  0.074151  0.062704  0.692604    41     4  26.399241
88  (0.4, 0.4)    0.5  0.074151  0.060136  0.747455     4     4   2.660162
89  (0.5, 0.5)    0.5  0.074151  0.061599  0.726080     4     4   2.540248

[90 rows x 8 columns]
         (p,r)  delta  RRE_mean   RRE_std  ACC_mean   ACC_std  NMI_mean  \
0 

## Stack NMF

In [14]:
Algo = 'StackedNMF'
Data = 'YaleB'
dataset = 'data/CroppedYaleB'
reduce = 3
X, Y = load_data(dataset, reduce=reduce)
seeds = [0,1,2,3,4]
pr_lst = [(0,0),(0.1,0.1),(0.2,0.2),(0.3,0.3),(0.4,0.4),(0.5,0.5)]
record_SNMF_Yale = []
layer_ranks=[10, 6]
steps_per_layer = 10000
tol = 1e-3
eps = 1e-9

for seed in seeds:
    np.random.seed(seed)
    for p, r in pr_lst:
        # X_noisy, noise = salt_pepper(X, p, r)
        X_noisy, Y = load_data_salt_pepper(dataset, reduce, p, r)
        start = time.time()
        H_final, PH, step = stackedNMF(X_noisy, layer_ranks=layer_ranks, steps_per_layer = steps_per_layer, tol = tol, eps=eps)
        end = time.time()
        timeusing = end - start
        rre = relative_reconstruction_error(X_noisy, PH)
        acc, nmi = evaluate_clustering(H_final, Y)
        record_SNMF_Yale.append(((p,r),acc,nmi,rre,tol,step,seed,timeusing))

Converged at step 77, e = 266.2816, rel_change=9.877e-04
Converged at step 54, e = 3.2216, rel_change=9.766e-04


D:\Software\Anaconda3\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1429: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=10.
  warnings.warn(


Acc(NMI) = 0.0667 (0.0389)
Converged at step 58, e = 613.2671, rel_change=9.944e-04
Converged at step 66, e = 2.5081, rel_change=9.427e-04
Acc(NMI) = 0.0646 (0.0397)


D:\Software\Anaconda3\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1429: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=10.
  warnings.warn(


Converged at step 43, e = 963.0315, rel_change=9.627e-04
Converged at step 91, e = 1.1991, rel_change=9.951e-04
Acc(NMI) = 0.0713 (0.0478)


...

D:\Software\Anaconda3\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1429: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=10.
  warnings.warn(


Converged at step 3, e = 1351.4633, rel_change=9.471e-04
Converged at step 67, e = 1.2455, rel_change=9.960e-04
Acc(NMI) = 0.0729 (0.0635)


D:\Software\Anaconda3\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1429: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=10.
  warnings.warn(


Converged at step 4, e = 1274.7925, rel_change=9.160e-04
Converged at step 73, e = 1.3101, rel_change=9.877e-04
Acc(NMI) = 0.0791 (0.0737)


D:\Software\Anaconda3\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1429: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=10.
  warnings.warn(


In [15]:
df_SNMF_Yale = pd.DataFrame(record_SNMF_Yale, columns=['(p,r)','acc','nmi','rre','tol','step','seed','time'])
print(df_SNMF_Yale)
summary = df_SNMF_Yale.groupby(['(p,r)']).agg(
    RRE_mean=('rre', 'mean'),
    RRE_std=('rre', 'std'),
    ACC_mean=('acc', 'mean'),
    ACC_std=('acc', 'std'),
    NMI_mean=('nmi', 'mean'),
    NMI_std=('nmi', 'std')
).reset_index()
print(summary)
df_SNMF_Yale.to_excel(f'{Algo}_{Data}.xlsx', index=True)

         (p,r)       acc       nmi       rre    tol  step  seed       time
0       (0, 0)  0.066694  0.038871  0.374514  0.001    55     0  18.243205
1   (0.1, 0.1)  0.064623  0.039698  0.521901  0.001    67     0  13.451988
2   (0.2, 0.2)  0.071251  0.047830  0.634970  0.001    92     0  10.755128
3   (0.3, 0.3)  0.073737  0.059138  0.747737  0.001    61     0   1.307281
4   (0.4, 0.4)  0.076222  0.053765  0.747035  0.001    79     0   1.314829
5   (0.5, 0.5)  0.073322  0.053129  0.725264  0.001    76     0   1.484611
6       (0, 0)  0.072080  0.064078  0.377293  0.001    42     1  18.211822
7   (0.1, 0.1)  0.066694  0.036753  0.518907  0.001    46     1  12.805039
8   (0.2, 0.2)  0.072080  0.049175  0.635479  0.001    96     1  12.431298
9   (0.3, 0.3)  0.074979  0.065816  0.747690  0.001    85     1   1.445989
10  (0.4, 0.4)  0.082850  0.073509  0.746978  0.001    64     1   1.326759
11  (0.5, 0.5)  0.074565  0.065014  0.725254  0.001    71     1   1.313086
12      (0, 0)  0.064623 

In [18]:
!pip install psutil

Looking in indexes: https://mirrors.aliyun.com/pypi/simple/



[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [19]:
import platform
import psutil
import datetime
import sys

def get_size(bytes, suffix="B"):
    """
    Scale bytes to its proper format (e.g., 1024 -> 1KB)
    """
    factor = 1024
    for unit in ["", "K", "M", "G", "T", "P"]:
        if bytes < factor:
            return f"{bytes:.2f}{unit}{suffix}"
        bytes /= factor

print("="*40, "System Information", "="*40)
uname = platform.uname()
print(f"Operating System: {uname.system} {uname.release}")
print(f"Version: {uname.version}")
print(f"Architecture: {uname.machine}")
print(f"Processor: {uname.processor}")
print(f"Hostname: {uname.node}")
print(f"Current Time: {datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

print("\n", "="*40, "CPU Information", "="*40)
print(f"Physical Cores: {psutil.cpu_count(logical=False)}")
print(f"Total Cores (Logical): {psutil.cpu_count(logical=True)}")
cpufreq = psutil.cpu_freq()
print(f"Max Frequency: {cpufreq.max:.2f}Mhz")
print(f"Current Frequency: {cpufreq.current:.2f}Mhz")

print("\n", "="*40, "Memory Information", "="*40)
svmem = psutil.virtual_memory()
print(f"Total: {get_size(svmem.total)}")
print(f"Used: {get_size(svmem.used)}")
print(f"Available: {get_size(svmem.available)}")
print(f"Percentage: {svmem.percent}%")

print("\n", "="*40, "Python Environment", "="*40)
print(f"Python Version: {sys.version}")
print(f"Jupyter Kernel: {platform.python_implementation()} {platform.python_version()}")

======================================== System Information ========================================
Operating System: Windows 11
Version: 10.0.26100
Architecture: AMD64
Processor: Intel64 Family 6 Model 183 Stepping 1, GenuineIntel
Hostname: wilsonng
Current Time: 2025-10-23 19:57:27

 ======================================== CPU Information ========================================
Physical Cores: 14
Total Cores (Logical): 20
Max Frequency: 2600.00Mhz
Current Frequency: 2600.00Mhz

 ======================================== Memory Information ========================================
Total: 23.73GB
Used: 15.05GB
Available: 8.68GB
Percentage: 63.4%

 ======================================== Python Environment ========================================
Python Version: 3.12.3 | packaged by conda-forge | (main, Apr 15 2024, 18:20:11) [MSC v.1938 64 bit (AMD64)]
Jupyter Kernel: CPython 3.12.3


In [20]:
import numpy
import pandas
import scipy
import matplotlib
import sklearn

print("="*40, "Key Library Versions", "="*40)
print(f"NumPy Version: {numpy.__version__}")
print(f"Pandas Version: {pandas.__version__}")
print(f"SciPy Version: {scipy.__version__}")
print(f"Matplotlib Version: {matplotlib.__version__}")
print(f"Scikit-learn Version: {sklearn.__version__}")

# Example for deep learning frameworks
try:
    import torch
    print(f"PyTorch Version: {torch.__version__}")
    if torch.cuda.is_available():
        print(f"CUDA Available: Yes")
        print(f"GPU Device: {torch.cuda.get_device_name(0)}")
except ImportError:
    print("PyTorch not installed.")

try:
    import tensorflow
    print(f"TensorFlow Version: {tensorflow.__version__}")
except ImportError:
    print("TensorFlow not installed.")

======================================== Key Library Versions ========================================
NumPy Version: 1.26.4
Pandas Version: 2.2.2
SciPy Version: 1.13.1
Matplotlib Version: 3.10.3
Scikit-learn Version: 1.5.1
PyTorch Version: 2.5.1
CUDA Available: Yes
GPU Device: NVIDIA GeForce RTX 4060 Laptop GPU
TensorFlow Version: 2.19.0
